In [15]:
import geopandas as gpd
import pandas as pd

# Caminho do shapefile
caminho = "data/PRODES_AMZ_Municipal_2023.shp"

# Ler o shapefile
gdf = gpd.read_file(caminho)

print("Colunas encontradas no shapefile:")
print(gdf.columns)


# Agrupar por estado e ano, somando área total desmatada
# Salvar como CSV

gdf.to_csv('data/desmatamento_por_estado.csv', index=False)

print("✅ Arquivo CSV agregado salvo com sucesso em data/desmatamento_por_estado.csv")



C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:198: RuntimeWarning: data/PRODES_AMZ_Municipal_2023.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Colunas encontradas no shapefile:
Index(['fid', 'state', 'path_row', 'main_class', 'class_name', 'sub_class',
       'def_cloud', 'julian_day', 'image_date', 'year', 'area_km', 'scene_id',
       'source', 'satellite', 'sensor', 'uuid', 'geometry'],
      dtype='object')
✅ Arquivo CSV agregado salvo com sucesso em data/desmatamento_por_estado.csv


In [13]:
import pandas as pd

url = "https://apisidra.ibge.gov.br/values/t/5938/n6/all/v/37/p/2021"
df = pd.read_json(url)
df.to_csv("data/pib_municipal_2023.csv", index=False)

print("✅ PIB municipal 2021 salvo com sucesso!")


✅ PIB municipal 2021 salvo com sucesso!


In [12]:
import pandas as pd
import os

# Caminho da base original
caminho = "data/desmatamento_por_estado.csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base
df = pd.read_csv(caminho)
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Manter apenas colunas importantes
colunas_importantes = [
    "fid",
    "state",
    "image_date",
    "area_km"
]
df = df[colunas_importantes]
print("🔹 Colunas mantidas:", list(df.columns))

df = df.rename(columns={
    "state": "UF"
})
# 3️⃣ Corrigir tipos de dados
df["area_km"] = pd.to_numeric(df["area_km"], errors="coerce")
df["image_date"] = pd.to_datetime(df["image_date"], errors="coerce")

# 4️⃣ Filtrar apenas registros de 2021 (sem criar coluna year permanente)
df = df.loc[df["image_date"].dt.year == 2021]

# 5️⃣ Remover linhas com dados inválidos
df = df.dropna(subset=["area_km", "image_date"])
# 6️⃣ Agregar total de área desmatada por estado
df_estado = df.groupby("UF", as_index=False)["area_km"].sum()

# 7️⃣ Salvar versão agregada
output = "data/limpos/desmatamento_2021_por_estado.csv"
df_estado.to_csv(output, index=False, encoding="utf-8-sig")

print("✅ Base agregada por estado salva com sucesso!")
print(df_estado)


✅ Base carregada com sucesso!
📊 Linhas: 801333
🔹 Colunas mantidas: ['fid', 'state', 'image_date', 'area_km']
✅ Base agregada por estado salva com sucesso!
   UF      area_km
0  AC   892.610973
1  AM  2167.098538
2  AP    16.089385
3  MA   318.482521
4  MT  1972.025765
5  PA  5085.511560
6  RO  1648.045256
7  RR   283.690960
8  TO    32.147452


In [11]:
import pandas as pd
import re
import os

# Caminho do arquivo original
caminho = "data/pib_municipal_2021.csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base de PIB municipal
df = pd.read_csv(caminho, encoding="utf-8")
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Renomear colunas relevantes
df = df.rename(columns={
    "V": "pib_mil_reais",
    "D1N": "municipio"
})

# 3️⃣ Extrair a sigla do estado a partir do nome do município
df["UF"] = df["municipio"].apply(
    lambda x: re.search(r"-\s*([A-Z]{2})$", x).group(1)
    if isinstance(x, str) and re.search(r"-\s*([A-Z]{2})$", x)
    else None
)

# 4️⃣ Converter valores para numérico
df["pib_mil_reais"] = pd.to_numeric(df["pib_mil_reais"], errors="coerce")

# 5️⃣ Filtrar apenas estados da Amazônia Legal
amazonia_legal = ["AC", "AM", "AP", "MA", "MT", "PA", "RO", "RR", "TO"]
df = df[df["UF"].isin(amazonia_legal)]

print("🌳 Estados filtrados (Amazônia Legal):", sorted(df["UF"].unique()))

# 6️⃣ Agrupar por estado e somar PIB
pib_estadual = df.groupby("UF", as_index=False)["pib_mil_reais"].sum()

# 7️⃣ Adicionar PIB em bilhões para facilitar leitura
pib_estadual["pib_bilhoes"] = (pib_estadual["pib_mil_reais"] / 1_000_000).round(2)

# 8️⃣ Salvar resultado
output = "data/limpos/pib_estadual_amazonia_2021.csv"
pib_estadual.to_csv(output, index=False, encoding="utf-8-sig")

print("✅ PIB estadual da Amazônia Legal salvo com sucesso!")
print(pib_estadual)


✅ Base carregada com sucesso!
📊 Linhas: 5571
🌳 Estados filtrados (Amazônia Legal): ['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']
✅ PIB estadual da Amazônia Legal salvo com sucesso!
   UF  pib_mil_reais  pib_bilhoes
0  AC     21374442.0        21.37
1  AM    131531039.0       131.53
2  AP     20099850.0        20.10
3  MA    124980727.0       124.98
4  MT    233390206.0       233.39
5  PA    262904976.0       262.90
6  RO     58170098.0        58.17
7  RR     18202580.0        18.20
8  TO     51780763.0        51.78


In [10]:
import pandas as pd
import os

# Caminho da base original
caminho = "data/br_ibge_populacao_municipio.csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base
df = pd.read_csv(caminho)
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Manter apenas colunas importantes
colunas_importantes = [
    "ano",
    "sigla_uf",
    "populacao"
]
df = df[colunas_importantes]
print("🔹 Colunas mantidas:", list(df.columns))

df = df.rename(columns={
    "sigla_uf": "UF"
})
# 3️⃣ Corrigir tipos de dados
df["ano"] = pd.to_numeric(df["ano"], errors="coerce")
df["populacao"] = pd.to_numeric(df["populacao"], errors="coerce")

# 4️⃣ Filtrar apenas registros de 2021 (sem criar coluna year permanente)
df = df.loc[df["ano"] == 2021]

# 5️⃣ Remover linhas com dados inválidos
df = df.dropna(subset=["populacao"])

amazonia_legal = ["AC", "AM", "AP", "MA", "MT", "PA", "RO", "RR", "TO"]
df= df[df["UF"].isin(amazonia_legal)]
# 6️⃣ Agregar total de área desmatada por estado
df_estado = df.groupby("UF", as_index=False)["populacao"].sum()


# 7️⃣ Salvar versão agregada
output = "data/limpos/br_ibge_populacao_estados_limpo.csv"
df_estado.to_csv(output, index=False, encoding="utf-8-sig")

print("✅ Base agregada por estado salva com sucesso!")
print(df_estado)


✅ Base carregada com sucesso!
📊 Linhas: 174388
🔹 Colunas mantidas: ['ano', 'sigla_uf', 'populacao']
✅ Base agregada por estado salva com sucesso!
   UF  populacao
0  AC   906876.0
1  AM  4269995.0
2  AP   877613.0
3  MA  7153262.0
4  MT  3567234.0
5  PA  8777124.0
6  RO  1815278.0
7  RR   652713.0
8  TO  1607363.0


In [9]:
import pandas as pd
import os

# Caminho da base original
caminho = "data/ipeadata[16-10-2025-10-43].csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base

df = pd.read_csv(
    caminho,
    skiprows=1
)
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Manter apenas colunas importantes
colunas_importantes = [
    "Sigla",
    "2021"
]
df = df[colunas_importantes]
print("🔹 Colunas mantidas:", list(df.columns))

df = df.rename(columns={
    "Sigla": "UF",
    "2021": "IDH"
})
# 3️⃣ Corrigir tipos de dados
df["IDH"] = pd.to_numeric(df["IDH"], errors="coerce")


# 5️⃣ Remover linhas com dados inválidos
df = df.dropna(subset=["IDH"])

amazonia_legal = ["AC", "AM", "AP", "MA", "MT", "PA", "RO", "RR", "TO"]
df= df[df["UF"].isin(amazonia_legal)]


# 7️⃣ Salvar versão agregada
output = "data/limpos/ipeadata_limpo.csv"
df.to_csv(output, index=False, encoding="utf-8-sig")

print("✅ Base agregada por estado salva com sucesso!")
print(df)


✅ Base carregada com sucesso!
📊 Linhas: 27
🔹 Colunas mantidas: ['Sigla', '2021']
✅ Base agregada por estado salva com sucesso!
    UF    IDH
0   AC  0.710
2   AM  0.700
3   AP  0.688
9   MA  0.676
12  MT  0.736
13  PA  0.690
20  RO  0.700
21  RR  0.699
26  TO  0.731


In [8]:
import pandas as pd
import os

# Caminho da pasta
base_path = "data/limpos/"

# Nomes dos arquivos
arquivos = {
    "populacao": "br_ibge_populacao_estados_limpo.csv",
    "desmatamento": "desmatamento_2021_por_estado.csv",
    "ipeadata": "ipeadata_limpo.csv",
    "pib": "pib_estadual_amazonia_2021.csv"
}

# Função auxiliar pra exibir infos resumidas
def mostrar_info(nome, df):
    print(f"\n=== 📄 {nome.upper()} ===")
    print("Colunas:", list(df.columns))
    print("Tipos:\n", df.dtypes)
    print("Amostra:")
    print(df.head(), "\n")

# Ler e exibir informações de cada base
bases = {}
for nome, arquivo in arquivos.items():
    caminho = os.path.join(base_path, arquivo)
    df = pd.read_csv(caminho)
    bases[nome] = df
    mostrar_info(nome, df)

# 🔎 Verificar colunas de identificação do estado
print("\n=== 🧭 Colunas de identificação (UF / Estado) ===")
for nome, df in bases.items():
    for col in df.columns:
        if "uf" in col.lower() or "estado" in col.lower():
            print(f"{nome}: '{col}'")

# 🔎 Verificar se todas as UFs estão na Amazônia Legal
amazonia_legal = {"AC","AM","AP","MA","MT","PA","RO","RR","TO"}
print("\n=== 🗺️ Estados presentes em cada base ===")
for nome, df in bases.items():
    uf_cols = [c for c in df.columns if "uf" in c.lower()]
    if uf_cols:
        print(f"{nome}: {sorted(set(df[uf_cols[0]].unique()) & amazonia_legal)}")

print("\n✅ Análise concluída!")



=== 📄 POPULACAO ===
Colunas: ['UF', 'populacao']
Tipos:
 UF            object
populacao    float64
dtype: object
Amostra:
   UF  populacao
0  AC   906876.0
1  AM  4269995.0
2  AP   877613.0
3  MA  7153262.0
4  MT  3567234.0 


=== 📄 DESMATAMENTO ===
Colunas: ['UF', 'area_km']
Tipos:
 UF          object
area_km    float64
dtype: object
Amostra:
   UF      area_km
0  AC   892.610973
1  AM  2167.098538
2  AP    16.089385
3  MA   318.482521
4  MT  1972.025765 


=== 📄 IPEADATA ===
Colunas: ['UF', 'IDH']
Tipos:
 UF      object
IDH    float64
dtype: object
Amostra:
   UF    IDH
0  AC  0.710
1  AM  0.700
2  AP  0.688
3  MA  0.676
4  MT  0.736 


=== 📄 PIB ===
Colunas: ['UF', 'pib_mil_reais', 'pib_bilhoes']
Tipos:
 UF                object
pib_mil_reais    float64
pib_bilhoes      float64
dtype: object
Amostra:
   UF  pib_mil_reais  pib_bilhoes
0  AC     21374442.0        21.37
1  AM    131531039.0       131.53
2  AP     20099850.0        20.10
3  MA    124980727.0       124.98
4  MT    23339

In [7]:

import pandas as pd

# Carregar todas as bases limpas
pop = pd.read_csv("data/limpos/br_ibge_populacao_estados_limpo.csv")
desm = pd.read_csv("data/limpos/desmatamento_2021_por_estado.csv")
idh = pd.read_csv("data/limpos/ipeadata_limpo.csv")
pib = pd.read_csv("data/limpos/pib_estadual_amazonia_2021.csv")

# Merge progressivo por 'UF'
df_final = pop.merge(desm, on="UF") \
              .merge(idh, on="UF") \
              .merge(pib, on="UF")

# Mostrar resultado
print("✅ Base final criada com sucesso!")
print(df_final.head())

# Salvar para uso no Scikit-Learn
df_final.to_csv("data/limpos/base_final.csv", index=False, encoding="utf-8-sig")


✅ Base final criada com sucesso!
   UF  populacao      area_km    IDH  pib_mil_reais  pib_bilhoes
0  AC   906876.0   892.610973  0.710     21374442.0        21.37
1  AM  4269995.0  2167.098538  0.700    131531039.0       131.53
2  AP   877613.0    16.089385  0.688     20099850.0        20.10
3  MA  7153262.0   318.482521  0.676    124980727.0       124.98
4  MT  3567234.0  1972.025765  0.736    233390206.0       233.39


In [16]:
import pandas as pd

# Carregar a base final
df = pd.read_csv("data/limpos/base_final.csv")

print("📊 Dimensões:", df.shape)
print("🔹 Colunas e tipos de dados:")
print(df.dtypes)
print("\n🔍 Valores nulos por coluna:")
print(df.isnull().sum())

# Conferir se todas as colunas numéricas são válidas
if df.select_dtypes(include=["float64", "int64"]).shape[1] == len(df.columns) - 1:
    print("\n✅ Todas as variáveis numéricas estão prontas pro Scikit-Learn!")
else:
    print("\n⚠️ Ainda há colunas não numéricas (verifique se o tipo 'object' aparece acima).")


📊 Dimensões: (9, 6)
🔹 Colunas e tipos de dados:
UF                object
populacao        float64
area_km          float64
IDH              float64
pib_mil_reais    float64
pib_bilhoes      float64
dtype: object

🔍 Valores nulos por coluna:
UF               0
populacao        0
area_km          0
IDH              0
pib_mil_reais    0
pib_bilhoes      0
dtype: int64

✅ Todas as variáveis numéricas estão prontas pro Scikit-Learn!
